# Qcodes example with DelegateInstrument driver for a device

This notebooks explains how to use the `DelegateInstrument` driver with both channels and parameters.

The goal of the `DelegateInstrument` driver is to make it easier to combine different parameters together into a new "virtual" instrument. Each parameter on a `DelegateInstrument` can point to one or more parameters on other instruments in the station. Each channel points to a channel of an instrument, i.e. a DAC, or can alternatively be instantiated with using a custom class.

The YAML file below specifies the instruments with the channels/parameters we wish to group into a new instrument, here called "device". The first example simply adds the channel 'as is' using `self.add_submodule`, while the parameter is added as a `DelegateParameter`. 

In [1]:
%%writefile example.yaml

instruments:
  lockin:
    type: qcodes.tests.instrument_mocks.MockLockin

  dac:
    type: qcodes.tests.instrument_mocks.MockDAC   

  device:
    type: qcodes.instrument.delegate.DelegateInstrument
    init:
      parameters:
        readout: lockin.X
      channels:
        gate_1: dac.ch01
      set_initial_values_on_load: true
      initial_values:
        readout: 1e-5
        gate_1.voltage.post_delay: 0.01

Overwriting example.yaml


In [2]:
import qcodes as qc

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /Users/jana/.qcodes/logs/command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : /Users/jana/.qcodes/logs/210608-2170-qcodes.log


In [3]:
station = qc.Station(config_file="example.yaml")

In [4]:
lockin = station.load_lockin()
dac = station.load_dac()
device = station.load_device(station=station)

In [5]:
print(device.gate_1)
print(device.gate_1.voltage.post_delay)

<MockDACChannel: dac_ch01 of MockDAC: dac>

In [6]:
device.gate_1.voltage()

0.0

The second example adds a channel using a custom channel class, which takes the initial channel and its name as input and has a parameter `current_valid_ranges`. 

In [7]:
%%writefile example.yaml

instruments:
  lockin:
    type: qcodes.tests.instrument_mocks.MockLockin

  dac:
    type: qcodes.tests.instrument_mocks.MockDAC   

  device:
    type: qcodes.instrument.delegate.DelegateInstrument
    init:
      parameters:
        readout: lockin.X
      channels:
        type: qcodes.tests.instrument_mocks.MockCustomChannel
        gate_1: 
          channel: dac.ch01
          current_valid_ranges: [-0.5, 0]
      set_initial_values_on_load: true
      initial_values:
        readout: 1e-5

Overwriting example.yaml


In [8]:
lockin.close()
dac.close()

In [9]:
station = qc.Station(config_file="example.yaml")
lockin = station.load_lockin()
dac = station.load_dac()

In [10]:
device = station.load_device(station=station)

In [11]:
device.gate_1

<MockCustomChannel: dac_gate_1 of MockDAC: dac>

In [12]:
device.gate_1.voltage(-0.3)

In [13]:
device.gate_1.voltage()

-0.3

In [14]:
device.gate_1.current_valid_range()

[-0.5, 0]